In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('hacker_clustering').getOrCreate()

In [2]:
from google.colab import files
uploaded = files.upload()

Saving hack_data.csv to hack_data.csv


In [4]:
df = spark.read.csv('hack_data.csv', header = True, inferSchema = True)

In [5]:
df.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [6]:
from pyspark.ml.clustering import KMeans

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
df.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [11]:
filterted_df = ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']

In [12]:
assembler = VectorAssembler(inputCols = filterted_df, outputCol = 'features')

In [13]:
final_df = assembler.transform(df)
final_df.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



In [14]:
from pyspark.ml.feature import StandardScaler

In [15]:
scaler = StandardScaler(inputCol = 'features', outputCol = 'scaledFeatures')

In [16]:
scaler_model = scaler.fit(final_df)
scaled_final_df = scaler_model.transform(final_df)

In [17]:
scaled_final_df.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaledFeatures: vector (nullable = true)



In [19]:
kmeans1 = KMeans(featuresCol = 'scaledFeatures', k = 2)
kmeans2 = KMeans(featuresCol = 'scaledFeatures', k = 3)

In [20]:
k2_model = kmeans1.fit(scaled_final_df)
k3_model = kmeans2.fit(scaled_final_df)

In [21]:
k3_model.transform(scaled_final_df).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         2|   79|
|         0|   88|
+----------+-----+



In [22]:
k2_model.transform(scaled_final_df).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

